# Instruction Fine-tuning sobre un LLM Base

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la realización de instruction fine-tuning sobre el LLM [Flan-T5-small](https://huggingface.co/google/flan-t5-small) con el objetivo de que sea capaz de resumir artículos de periódico en el idioma Español.

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Importamos el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importamos el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Generación de texto

In [3]:
text = """Astrónomos detectaron una misteriosa ráfaga de ondas de radio que tardó \
8.000 millones de años en llegar a la Tierra. La ráfaga rápida de radio es una de \
las más distantes y energéticas jamás observadas. Las ráfagas rápidas de radio \
(FRB, por sus siglas en inglés) son intensos estallidos de ondas de radio de \
milisegundos de duración cuyo origen se desconoce. La primera FRB se descubrió \
en 2007 y, desde entonces, se han detectado cientos de estos rápidos destellos \
cósmicos procedentes de puntos distantes de todo el universo."""

In [4]:
text = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [5]:
text = """"El telescopio Hubble, lanzado al espacio en 1990, ha proporcionado imágenes \
impresionantes del universo y ha ayudado a los científicos a comprender mejor la cosmología."""

In [6]:
prompt_template = f"Resume el siguiente articulo:\n\n{text}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5.generate(prompt_tokens, max_length=200)

# Transformamos los tokens generados en texto
print(tokenizer_FT5.decode(outputs[0]))

<pad> Hubble, a telescopio in space in 1990, has provided impressive images of the universe and has helped scientists to understand better the Cosmology.</s>


## 2. Selección y preparación del conjunto de datos

El conjunto de datos que vamos a utilizar para la realización del fine-tuning se corresponde con MLSUM.

MLSUM es el primer conjunto de datos de resumen multilingüe a gran escala. Obtenido de periódicos en línea, contiene más de 1.5 millones de pares de artículo/resumen en cinco idiomas diferentes: francés, alemán, español, ruso y turco.

La estructura de los datos es la siguiente:

```
{
    "date": "01/01/2001",
    "summary": "A text",
    "text": "This is a text",
    "title": "A sample",
    "topic": "football",
    "url": "https://www.google.com"
}
```

Para más información: https://huggingface.co/datasets/mlsum

### 2.1. Lectura del conjunto de datos

In [7]:
from datasets import load_dataset

ds = load_dataset("mlsum", 'es')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mlsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/266367 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10358 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13920 [00:00<?, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 266367
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 10358
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 13920
    })
})

In [9]:
# Mostramos un ejemplo del subconjunto de datos de entrenamiento
ds["train"]["text"][10]

'España ha sumado en 2009 un sexto año consecutivo de reducción de la mortalidad en las carreteras y, a falta de las cifras oficiales que confirmará hoy el ministro del Interior, Alfredo Pérez Rubalcaba, por primera vez en la historia se ha cerrado un periodo de 12 meses por debajo de 2.000 muertos. Hasta el pasado 18 de diciembre, último día del que se dispone de datos, la cifra de fallecidos es de 1.826, a los que hay que sumar los 31 de la primera fase de la operación especial de Navidad, entre el 22 y el 27 de diciembre. La estadística resulta especialmente llamativa si se compara con el primer año en el que se hizo balance, 1969, en el que 3.951 personas perdieron la vida. Desde entonces, el pico más alto se alcanzó en 1989 con 7.000 víctimas. En las cifras de mortalidad de 2009 destacan otros aspectos positivos como el hecho de que sólo se han superado los 200 muertos en un mes (agosto) y que ha habido cinco días sin víctimas mortales sobre el asfalto, el último el pasado 11 de d

In [10]:
# Mostramos el resumen correspondiente al ejemplo anterior
ds["train"]["summary"][10]

'2009 es el periodo con menos fallecidos en accidentes de tráfico en cuatro décadas, desde que existen datos oficiales'

### 2.2. Formato del conjunto de datos

Tal y como hemos comentado en secciones anteriores, el conjunto de datos utilizado para aplicar instruction fine-tuning debe estar formado por ejemplos de entrenamiento de la siguiente forma:
```
(prompt, completion)
```
Para formar el prompt debemos tener en cuenta los siguientes puntos:


*   Debe indicarse una instrucción para que realice el modelo. Es habitual utilizar plantillas que proponen los desarrolladores de los LLM para diseñar nuestros ejemplos de entrenamiento: https://github.com/google-research/FLAN/blob/main/flan/v2/flan_templates_branched.py

La plantilla que vamos a seleccionar es la siguiente:


```
("Resume el siguiente articulo:\n\n{text}", "{summary}")
```



*   Para afinar modelos como FLAN-T5 en tareas conversacionales donde queramos preservar el contexto de la conversación, se adopta un enfoque basado en secuencias, donde la interacción entre los participantes de la conversación se estructura en una sola cadena. La pregunta y la respuesta suelen estar separadas por un token especial, como <sep>, <eos>, o simplemente utilizando un delimitador claro (ej: `\n`)



```
("Conversación:\n[Usuario] ¿Cuál es la capital de Francia?\n[Asistente] La capital de Francia es París.\n[Usuario] ¿Y cuál es su río principal?\n[Asistente] ", "Su río principal es el Sena")
```




Reducimos el tamaño del conjunto de datos para consumir menos recursos computacionales

In [11]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 1500
NUM_EJ_VAL = 500
NUM_EJ_TEST = 200

# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['validation'] = ds['validation'].select(range(NUM_EJ_VAL))

# Subconjunto de pruebas
ds['test'] = ds['test'].select(range(NUM_EJ_TEST))

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 500
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 200
    })
})

Pre-procesamos el conjunto de datos para aplicar la plantilla seleccionada anteriormente.

In [13]:
def parse_dataset(ejemplo):
  """Procesa los ejemplos para adaptarlos a la plantilla."""
  return {"prompt": f"Resume el siguiente articulo:\n\n{ejemplo['text']}"}

In [14]:
ds["train"] = ds["train"].map(parse_dataset)
ds["validation"] = ds["validation"].map(parse_dataset)
ds["test"] = ds["test"].map(parse_dataset)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt'],
        num_rows: 500
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt'],
        num_rows: 200
    })
})

In [16]:
print(ds["train"]["prompt"][10])

Resume el siguiente articulo:

España ha sumado en 2009 un sexto año consecutivo de reducción de la mortalidad en las carreteras y, a falta de las cifras oficiales que confirmará hoy el ministro del Interior, Alfredo Pérez Rubalcaba, por primera vez en la historia se ha cerrado un periodo de 12 meses por debajo de 2.000 muertos. Hasta el pasado 18 de diciembre, último día del que se dispone de datos, la cifra de fallecidos es de 1.826, a los que hay que sumar los 31 de la primera fase de la operación especial de Navidad, entre el 22 y el 27 de diciembre. La estadística resulta especialmente llamativa si se compara con el primer año en el que se hizo balance, 1969, en el que 3.951 personas perdieron la vida. Desde entonces, el pico más alto se alcanzó en 1989 con 7.000 víctimas. En las cifras de mortalidad de 2009 destacan otros aspectos positivos como el hecho de que sólo se han superado los 200 muertos en un mes (agosto) y que ha habido cinco días sin víctimas mortales sobre el asfalt

In [17]:
print(ds["train"]["text"][10])

España ha sumado en 2009 un sexto año consecutivo de reducción de la mortalidad en las carreteras y, a falta de las cifras oficiales que confirmará hoy el ministro del Interior, Alfredo Pérez Rubalcaba, por primera vez en la historia se ha cerrado un periodo de 12 meses por debajo de 2.000 muertos. Hasta el pasado 18 de diciembre, último día del que se dispone de datos, la cifra de fallecidos es de 1.826, a los que hay que sumar los 31 de la primera fase de la operación especial de Navidad, entre el 22 y el 27 de diciembre. La estadística resulta especialmente llamativa si se compara con el primer año en el que se hizo balance, 1969, en el que 3.951 personas perdieron la vida. Desde entonces, el pico más alto se alcanzó en 1989 con 7.000 víctimas. En las cifras de mortalidad de 2009 destacan otros aspectos positivos como el hecho de que sólo se han superado los 200 muertos en un mes (agosto) y que ha habido cinco días sin víctimas mortales sobre el asfalto, el último el pasado 11 de di

### 2.3. Tokenización del conjunto de datos

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

Una de las cosas que comentabamos cuándo comenzamos a hablar de modelos generativos como la Redes Neuronales Recurrentes, es que este tipo de algoritmos, al igual que los LLMs, reciben secuencias del mismo tamaño.

Con lo cual, al igual que hicimos en ese caso práctico al comienzo del curso, debemos obtener la secuencia más larga de nuestro conjunto de datos y realizar padding al resto de secuencias para que todas tengan el mismo tamaño.

In [19]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True) # Va a truncar en 512 que es el tamaño máximo para este modelo
max_token_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_token_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Maximo tamaño de prompt: 512


Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Maximo tamaño de completion: 242


In [20]:
def padding_tokenizer(datos):
  # Tokenizar inputs (prompts)
  model_inputs = tokenizer(datos['prompt'], max_length=max_token_len, padding="max_length", truncation=True)

  # Tokenizar labels (completions)
  model_labels = tokenizer(datos['summary'], max_length=max_completion_len, padding="max_length", truncation=True)

  # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
  model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

  model_inputs['labels'] = model_labels["input_ids"]

  return model_inputs

In [21]:
ds_tokens = ds.map(padding_tokenizer, batched=True, remove_columns=['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt'])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [22]:
ds_tokens

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [23]:
ds_tokens["train"]["input_ids"][10]

[9410,
 3,
 15,
 40,
 108,
 7938,
 4617,
 3,
 12277,
 32,
 10,
 28774,
 2,
 9,
 4244,
 4505,
 9,
 26,
 32,
 3,
 35,
 2464,
 73,
 3,
 7,
 994,
 235,
 3,
 9,
 2,
 32,
 6900,
 15,
 3044,
 23,
 1621,
 20,
 27353,
 12765,
 20,
 50,
 24301,
 15644,
 3,
 35,
 50,
 7,
 443,
 60,
 449,
 9,
 7,
 3,
 63,
 6,
 3,
 9,
 12553,
 17,
 9,
 20,
 50,
 7,
 3,
 31812,
 7,
 11343,
 15,
 7,
 238,
 3606,
 291,
 2975,
 3534,
 63,
 3,
 15,
 40,
 3016,
 6626,
 20,
 40,
 8226,
 6,
 19850,
 32,
 276,
 154,
 2638,
 15612,
 138,
 10891,
 9,
 6,
 5569,
 21628,
 9,
 3,
 6071,
 3,
 35,
 50,
 3,
 107,
 17905,
 142,
 4244,
 3,
 2110,
 19042,
 73,
 1059,
 32,
 20,
 586,
 140,
 2260,
 5569,
 20,
 115,
 9,
 1927,
 20,
 3,
 26756,
 4035,
 49,
 235,
 7,
 5,
 4498,
 17,
 9,
 3,
 15,
 40,
 330,
 9,
 26,
 32,
 507,
 20,
 3,
 26,
 1294,
 21388,
 6,
 3,
 2,
 40,
 2998,
 32,
 3,
 26,
 2,
 9,
 20,
 40,
 238,
 142,
 3,
 10475,
 782,
 20,
 3927,
 32,
 7,
 6,
 50,
 3,
 31812,
 20,
 1590,
 15,
 75,
 28594,
 3,
 15,
 7,
 20,
 3,
 16253,


## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [24]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 3.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [25]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 3.3. Lectura y adaptación de los datos para el entrenamiento

In [26]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### Preparación y ejecución del fine-tuning (entrenamiento)

In [27]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=4,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_tokens["train"],
    eval_dataset=ds_tokens["validation"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{REPOSITORY}/tokenizer")

('/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/spiece.model',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/tokenizer.json')

In [29]:
# Iniciamos el entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.259183,15.049700,2.852300,13.050500,13.123100,18.990000
2,No log,2.252859,15.392300,2.891100,13.245100,13.350900,18.998000
3,2.715600,2.246323,14.955500,2.704700,12.987300,13.106300,19.000000
4,2.715600,2.243661,14.760200,2.651800,12.808500,12.934600,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=752, training_loss=2.6784683389866606, metrics={'train_runtime': 558.7859, 'train_samples_per_second': 10.738, 'train_steps_per_second': 1.346, 'total_flos': 1115343028224000.0, 'train_loss': 2.6784683389866606, 'epoch': 4.0})

## 4. Generación de texto con Flan-T5 Fine-tuned y evaluación

### Lectura del modelo y del tokenizador

In [30]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Importamos el tokenizador
tokenizer_FT5_FT = T5Tokenizer.from_pretrained(f"{REPOSITORY}/tokenizer")

# Importamos el modelo con fine-tuning
model_FT5_FT = T5ForConditionalGeneration.from_pretrained(f"{REPOSITORY}/checkpoint-752", device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Generación de texto

In [31]:
text = """Astrónomos detectaron una misteriosa ráfaga de ondas de radio que tardó \
8.000 millones de años en llegar a la Tierra. La ráfaga rápida de radio es una de \
las más distantes y energéticas jamás observadas. Las ráfagas rápidas de radio \
(FRB, por sus siglas en inglés) son intensos estallidos de ondas de radio de \
milisegundos de duración cuyo origen se desconoce. La primera FRB se descubrió \
en 2007 y, desde entonces, se han detectado cientos de estos rápidos destellos \
cósmicos procedentes de puntos distantes de todo el universo."""

In [32]:
text = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [33]:
text = """"El telescopio Hubble, lanzado al espacio en 1990, ha proporcionado imágenes \
impresionantes del universo y ha ayudado a los científicos a comprender mejor la cosmología."""

In [34]:
# Construimos el prompt conforme a la plantilla de fine-tuning
prompt_template = f"Resume el siguiente articulo:\n\n{text}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

# Transformamos los tokens generados en texto
print(tokenizer_FT5_FT.decode(outputs[0]))

<pad> Hubble proporciona una imágene impresionante del universo y ayudada a los cient<unk> ficos a comprender la cosmolog<unk> a</s>


### Evaluación con el subconjunto de pruebas

In [35]:
import torch

# Cambiamos el modelo al modo de evaluación
model_FT5_FT.eval()

# Definir tamaño del lote
batch_size = 8

all_predictions = []

# Deshabilitamos el entrenamiento y obtenemos las completions
with torch.no_grad():
  for i in range(0, len(ds_tokens["test"]["input_ids"]), batch_size):
        # Extraemos el lote actual
        input_ids_batch = torch.tensor(ds_tokens["test"]["input_ids"][i:i+batch_size], device='cuda:0')

        # Obtenemos las predicciones del modelo
        outputs = model_FT5_FT.generate(input_ids_batch)

        # Concatenemos las predicciones
        all_predictions.extend(outputs)

# Calculamos las metricas
labels = np.array(ds_tokens['test']['labels'])
completions = np.array([pred.cpu().numpy() for pred in all_predictions])

metrics = compute_metrics((completions, labels))

print(metrics)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 13.8489, 'rouge2': 2.3453, 'rougeL': 11.8577, 'rougeLsum': 11.9538, 'gen_len': 18.985}
